In [1]:
!nvidia-smi

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Wed Sep 13 11:00:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.05              Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  | 00000000:01:00.0 Off |                  N/A |
|  0%   32C    P8              10W / 220W |    479MiB /  8192MiB |      0%      Default |
|                                         |                      |                  

In [2]:
pwd

'/tmp/rjUq5CpGra'

# 3. 뉴스 요약봇 만들기

## 3-1. 들어가며

### 학습 내용
2. 텍스트 요약(Text Summarization)
 - 텍스트 요약 방법인 추출적 요약(Extractive Summarization)과 추상적 요약(Abstractive Summarization)에 대해서 알아봅니다.
3. 인공 신경망으로 텍스트 요약 훈련시키기
 - seq2seq 모델에 대한 개요와 구조 그리고 요소들에 대해서 알아봅니다.
4. 데이터 준비하기
 - Kaggle에서 제공하는 아마존 리뷰 데이터셋을 다운받고, 데이터를 확인해 봅니다.
5.  ~ 7. 데이터 전처리하기
 - 불용어 제거, 정규화, 정수인코딩 등의 데이터 전처리 과정을 코드로 구현합니다.
8. 모델 설계하기
 - 인코더와 디코더, 어텐셔을 설계하고 코드로 구현합니다.
9. 모델 훈련하기
 - EarlyStopping에 대해서 알아보고, 이를 적용하여 모델을 학습합니다.
10. 인퍼런스 모델 구현하기
 - 정수 인덱스 행렬로 나온 결과값을 실제 데이터로 복원하는 인퍼런스 모델을 코드로 구현합니다.
11. 모델 테스트하기
 - 모델을 통해 얻은 요약문과 실제 요약문을 비교해 봅니다.
12. 추출적 요약 해보기
 - summa 패키지를 사용하여 추출적 요약(Extractive Summarization)을 해봅니다.

### 학습 목표

Extractive/Abstractive summarization 이해할 수 있습니다.

단어장 크기를 줄이는 다양한 text normalization 적용할 수 있습니다.

seq2seq의 성능을 Up시키는 Attention Mechanism 적용할 수 있습니다.

## 3-2. 텍스트 요약(Text Summarization)이란?

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/E-21-1.png)
텍스트 요약(Text Summarization)이란 위 그림과 같이 긴 길이의 문서(Document) 원문을 핵심 주제만으로 구성된 짧은 요약(Summary) 문장들로 변환하는 것을 말합니다. 예를 들어 상대적으로 큰 텍스트인 뉴스 기사로 작은 텍스트인 뉴스 제목을 만들어내는 것이 텍스트 요약의 대표적인 예라고 할 수 있어요.

이때 중요한 것은 요약 전후에 정보 손실 발생이 최소화되어야 한다는 점입니다. 이것은 정보를 압축하는 과정과 같습니다. 비록 텍스트의 길이가 크게 줄어들었지만, 요약문은 문서 원문이 담고 있는 정보를 최대한 보존하고 있어야 합니다. 이것은 원문의 길이가 길수록 만만치 않은 어려운 작업이 됩니다. 사람이 이 작업을 수행한다 하더라도 긴 문장을 정확하게 읽고 이해한 후, 그 의미를 손상하지 않는 짧은 다른 표현으로 원문을 번역해 내야 하는 것입니다.

그렇게 요약 문장을 만들어 내려면 어떤 방법을 사용하면 좋을까요? 여기서 텍스트 요약은 크게 추출적 요약(Extractive Summarization)과 추상적 요약(Abstractive Summarization)의 두 가지 접근으로 나누어볼 수 있습니다.

### 추출적 요약(Extractive Summarization)
첫 번째 방식인 추출적 요약은 단어 그대로 원문에서 문장들을 추출해서 요약하는 방식이에요. 가령, 10개의 문장으로 구성된 텍스트가 있다면, 그중 핵심적인 문장 3개를 꺼내와서 3개의 문장으로 구성된 요약문을 만드는 식이죠. 그런데 꺼내온 3개의 문장이 원문에서 중요한 문장일 수는 있어도, 3개의 문장의 연결이 자연스럽지 않을 수는 있거든요. 결과로 나온 문장들 간의 호응이 자연스럽지 않을 수 있다는 것이죠. 딥 러닝보다는 주로 전통적인 머신 러닝 방식에 속하는 텍스트 랭크(TextRank)와 같은 알고리즘을 사용해서 이 방법을 사용한다고 해요.

이런 방식을 이미 서비스에 도입해서 활용하고 있는 사례가 있다는 것, 알고 계셨나요? 가장 대표적인 것이 네이버 뉴스 서비스에 있는 요약봇 기능입니다.

네이버 뉴스에 접속해서 아무 뉴스 기사나 클릭해 봅시다. 제목 우하단의 요약봇 버튼을 다시 클릭해 보세요. 기사 원문을 단 3줄로 요약한 글을 보실 수 있습니다. 어떤가요? 가끔은 세 문장 간 연결이 조금 매끄럽지 않게 느껴질 때도 있지만 꽤 그럴듯한 요약문으로 보입니다. 위에서 소개한 TextRank 알고리즘을 통해 해당 기사를 가장 잘 대표하는 단어들로 이루어진 핵심문장을 아주 효과적으로 찾아내기 때문입니다. 잘 찾아보면 요약문에 사용된 문장 3개가 원문에 그대로 있다는 것을 알 수 있을 것입니다.

### 추상적 요약(Abstractive Summarization)
두 번째 방식인 추상적 요약은 추출적 요약보다 좀 더 흥미로운 접근을 사용합니다. 원문으로부터 내용이 요약된 새로운 문장을 생성해내는 것이죠. 여기서 새로운 문장이라는 것은 결과로 나온 문장이 원문에 원래 없던 문장일 수도 있다는 것을 의미합니다. 자연어 처리 분야 중 자연어 생성(Natural Language Generation, NLG)의 영역인 셈이죠. 반면, 추출적 요약은 원문을 구성하는 문장 중 어느 것이 요약문에 들어갈 핵심문장인지를 판별한다는 점에서 문장 분류(Text Classification) 문제로 볼 수 있을 것입니다.

자연어 생성하면 혹시 떠오르는 신경망이 있나요? 가장 기본적인 신경망 중 하나인데... 맞아요! RNN으로 이 문제를 풀 수 있겠군요! 그렇다면, RNN으로 추상적 요약 방식을 구현한다고 하면 문제가 전혀 없을까요?

Q. RNN은 학습 데이터의 길이가 길어질수록 먼 과거의 정보를 현재에 전달하기 어렵다는 문제가 있습니다. 이 문제를 해결하기 위해 LSTM과 GRU가 등장했고, 이 둘도 부족해서 어텐션(Attention) 메커니즘이 등장했지요. 이 문제의 이름은 무엇인가요?

장기 의존성(long term dependencies) 문제

장기 의존성 문제는 순환 신경망(RNN)에서 발생하는 문제로, 학습 데이터의 길이가 길어질수록 먼 과거의 정보를 현재에 전달하기 어렵다는 것을 의미합니다. RNN은 시간적인 의존성을 모델링하는 데 강점이 있지만, 그래디언트 소실 또는 그래디언트 폭주와 같은 문제로 인해 장기적인 의존성을 적절히 학습하기 어려울 수 있습니다.

장기 의존성 문제를 해결하기 위해 등장한 LSTM(Long Short-Term Memory)과 GRU(Gated Recurrent Unit)은 RNN의 변형 구조로, 기본 RNN보다 더 긴 시간 간격에서의 정보를 보다 효과적으로 전달할 수 있도록 설계되었습니다. LSTM과 GRU는 게이트 메커니즘이라는 구조를 도입하여 그래디언트 흐름을 제어하고 장기적인 의존성을 캡처할 수 있는 기능을 제공합니다.

그러나 LSTM과 GRU도 모든 상황에서 완벽한 해결책은 아니며, 특히 입력 시퀀스가 매우 긴 경우에도 여전히 한계가 있습니다. 이에 따라 어텐션 메커니즘이 등장하였습니다. 어텐션은 입력 시퀀스의 다양한 위치에 주목(attention)하여 중요한 정보를 집중(concentrate)시키고, 필요한 정보에 가중치를 부여하여 모델이 장기 의존성을 보다 효과적으로 학습할 수 있게 돕습니다.

그저 RNN을 이용해 Language Generation을 한다고 해서 긴 문장을 읽고 나서 요약문을 뚝딱 만들어내긴 어렵겠죠. 어떤 방법을 사용할 지 고민을 할 때, 잘 하는 친구들은 어떻게 하는지 찾아보는 것은 꽤 많은 도움이 돼요. ML 분야의 선구자라고 할 수 있는 기업 '구글(Google)'은 자신들의 서비스에 어떤 방식으로 텍스트 요약을 시도했었을까요? 2016년의 아래 기사에 따르면, 구글은 뉴스 기사 내용으로부터 자동으로 뉴스 제목을 뽑아내는 텍스트 요약 모델을 구현했었다고 합니다. 어떻게 구현했는지, 잠깐 읽어볼까요?


구글 인공지능 "뉴스 제목도 잘 뽑네"

https://zdnet.co.kr/view/?no=20160905114833&from=Mobile

Q. 구글에서 텍스트 요약을 위해 시도했던 접근법 중에, 텍스트마이닝 분야의 '역문서빈도(IDF)같은' 지표를 활용해 문서 안에서 중요해 보이는 부분을 추출하고 그걸 요약문에 담는 방식을 썼을 때의 문제점은 무엇이었나요?

원문에서 발췌하는 방식(Extractive summarization)의 요약 기법은 어색하거나 문법적으로 이상한 결과물을 만드는 문제가 있음

Q. 구글은 짧은 문장. 요약문을 생성하는 모델을 딥 러닝을 통해 end-to-end로 설계하도록 했어요. 구글이 메일서비스에 적용한 자동 회신(Smart Reply) 기능을 만든 것과 비슷한 딥러닝 기법이기도 한 인코더와 디코더의 구조로 구성된 이 딥 러닝 아키텍처의 이름은 무엇일까요?

구글이 메일서비스에 적용한 자동 회신(Smart Reply) 기능과 유사한 딥러닝 기법으로 구성된 딥 러닝 아키텍처의 이름은 "Sequence-to-Sequence (Seq2Seq) 모델"입니다. Seq2Seq 모델은 인코더와 디코더라는 두 개의 주요 구성 요소로 이루어져 있습니다.

인코더는 입력 시퀀스(예: 원본 문장)를 고정된 크기의 잠재 벡터(latent vector)로 인코딩하는 역할을 합니다. 이 잠재 벡터는 입력 시퀀스의 의미와 정보를 담고 있습니다.

디코더는 인코더에서 얻은 잠재 벡터를 기반으로 출력 시퀀스(예: 요약문)를 생성하는 역할을 합니다. 디코더는 시작 토큰을 입력으로 받아서 한 번에 하나의 토큰을 생성하며, 이전에 생성된 토큰들과 현재 상태를 사용하여 다음 토큰을 예측합니다. 이런 식으로 디코더가 한 단어(또는 문자) 씩 출력하여 최종적으로 요약문을 완성합니다.

Seq2Seq 모델은 번역, 대화 생성, 요약 등 다양한 자연어 처리 작업에서 활용되며, 구글의 Smart Reply와 같이 짧은 문장에 대한 요약/응답 생성에도 사용됩니다.

## 3-3. 인공 신경망으로 텍스트 요약 훈련시키기

우리는 seq2seq 모델을 통해서 Abstractive summarization 방식의 텍스트 요약기를 만들어볼 거예요. seq2seq은 두 개의 RNN 아키텍처를 사용하여 입력 시퀀스로부터 출력 시퀀스를 생성해 내는 자연어 생성 모델입니다. 주로 뉴럴 기계번역에 사용되는 이 모델이 텍스트 요약에도 사용될 수 있을지 갸우뚱하실 수도 있겠지만, 원문을 요약문으로 번역한다고 생각한다면 전혀 무리가 없겠죠?

seq2seq 개요

![](https://d3s0tskafalll9.cloudfront.net/media/images/E-21-2.max-800x600.png)

https://medium.com/dl-for-product-and-service/abstractive-text-summary-with-reinforcement-learning-ab2458ab29d5

원문을 첫 번째 RNN인 인코더로 입력하면, 인코더는 이를 하나의 고정된 벡터로 변환해요. 이 벡터를 문맥 정보를 가지고 있는 벡터라고 하여 컨텍스트 벡터(context vector)라고 합니다. 두 번째 RNN인 디코더는 이 컨텍스트 벡터를 전달받아 한 단어씩 생성해내서 요약 문장을 완성하는 거죠.

###  LSTM과 컨텍스트 벡터

우리는 seq2seq를 구현할 때, 인코더/디코더로 바닐라 RNN이 아니라 LSTM을 사용할 거예요.

![](https://d3s0tskafalll9.cloudfront.net/media/images/E-21-3.max-800x600.png)

[RNN과 LSTM]
http://colah.github.io/posts/2015-08-Understanding-LSTMs

LSTM이 바닐라 RNN과 다른 점은 다음 time step의 셀에 hidden state뿐만 아니라, cell state도 함께 전달한다는 점이에요. 다시 말해, 인코더가 디코더에 전달하는 컨텍스트 벡터 또한 hidden state h와 cell state c 두 개의 값 모두 존재해야 한다는 뜻이죠.

### 시작 토큰과 종료 토큰

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/E-21-4.png)

[시작 토큰 SOS와 종료 토큰 EOS는 각각 start of a sequence와 end of a sequence를 나타낸다]
https://arxiv.org/pdf/1812.02303.pdf

seq2seq 구조에서 디코더는 시작 토큰 SOS가 입력되면, 각 시점마다 단어를 생성하고 이 과정을 종료 토큰 EOS를 예측하는 순간까지 멈추지 않아요. 다시 말해 훈련 데이터의 예측 대상 시퀀스의 앞, 뒤에는 시작 토큰과 종료 토큰을 넣어주는 전처리를 통해 어디서 멈춰야 하는지 알려줄 필요가 있겠죠.